In [12]:
from datasets import load_dataset
import random

model_name = "cyberagent/calm3-22b-chat"

In [2]:
ds=load_dataset("globis-university/aozorabunko-clean",split="train")

Generating train split: 100%|██████████| 16951/16951 [00:04<00:00, 3834.57 examples/s]


In [9]:
def extract_random_text(ds):
    record=ds[random.randint(0,len(ds))]
    extract_length=random.randint(100,300)
    extract_pos=random.randint(0,len(record["text"])-extract_length)
    extracted_text=record["text"][extract_pos:extract_pos+extract_length].strip()
    return extracted_text

In [ ]:
#auto reload modules
%load_ext autoreload
%autoreload 2

from datetime import datetime
import os
import random
import json
from vllm import LLM, SamplingParams
pid = os.getpid()
seed = int(pid)+int(datetime.now().timestamp())
print("seed: ", seed)
random.seed(seed)



out_dir = "data"
os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")



In [ ]:

def llm_gen(model, prompt_list,
            temperature=0.7, top_k=50,
            max_tokens=1024,
            ):
    outputs = model.generate(
        prompt_list,
        sampling_params=SamplingParams(
            temperature=temperature,
            max_tokens=max_tokens,
            top_k=top_k,
        )
    )
    return [i.outputs[0].text.strip() for i in outputs]


tensor_parallel_size = 1
# トークナイザーとモデルの準備
model = LLM(
    model=model_name,
    trust_remote_code=True,
    max_model_len=2000,
    tensor_parallel_size=tensor_parallel_size,
)



In [13]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [14]:


genres = [
    "心温まる日常会話",
    "楽しい会話",
    "ユーモラスな会話",
    "日常会話",
    "傾聴の会話",
    "相手の興味をそそる会話",
]


def clean_output(text):
    if text.startswith("「"):
        text = text[1:]
    if text.endswith("」"):
        text = text[:-1]
    return text


In [30]:
import copy
def ask_model(messages,model):
    messages=copy.deepcopy(messages)

    if messages[-1]["role"]=="user":
        messages.append({"role": "assistant", "content": ""})
    else:
        messages.append({"role": "user", "content": ""})
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False,)
    prompt=prompt.replace("<|im_end|>\n","")
    text=llm_gen(model, [prompt], temperature=0.7, top_k=50, max_tokens=1024)[0]
    return text

def gen_init_text(line, genre):
    system_list = [
        f"次の文章をもとに､{genre}のはじめのセリフを出力しなさい｡対話はタメ口で行われ､敬語は使いません｡セリフのみを出力し、それ以外は何も出力しないでください｡",
        f"次の文章をもとに､{genre}のはじめのセリフを出力しなさい｡対話は敬語で行われます｡セリフのみを出力し、それ以外は何も出力しないでください｡",
    ]
    system_message = random.choice(system_list)

    prompt = f"""
    #参考にする文章: {line}
    """

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]
    text=ask_model(messages,model)
    return text


In [ ]:
def gen_text( genres, client,
             n_replies=8):
    genre = random.choice(genres)

    seed_text=extract_random_text(ds)
    text = gen_init_text(seed_text, genre)

    system_list = [
        f"{genre}の返答文を生成しなさい｡セリフのみを出力し、それ以外は何も出力しないでください｡",
    ]
    system_message = random.choice(system_list)
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": text}
    ]

    for j in range(n_replies):
        res = ask_model(messages, client)
        if j % 2 == 1:
            messages.append({"role": "user", "content": res})
        else:
            messages.append({"role": "assistant", "content": res})

    return messages



In [31]:
seed_text=extract_random_text(ds)
prompt=gen_init_text(seed_text,random.choice(genres))

print(prompt)

<|im_start|>system
次の文章をもとに､心温まる日常会話のはじめのセリフを出力しなさい｡対話は敬語で行われます｡セリフのみを出力し、それ以外は何も出力しないでください｡<|im_start|>user

    #参考にする文章: ○帽子は既に述べしが如く洋服の形に従つて各戴くべきものあり。背広に鳥打帽を冠るは適しからず。鳥打帽はその名の如く銃猟、旅行航海等の折にのみ用るものにて、平生都会にてこれを戴くもの巴里あたりにては職工か新聞売子なぞなるべし。欧米ともに黒の山高帽は普通一般に用ひらるるものなり。殊に米国東部の都市にては晴雨共に風甚しきが故、中折帽は吹飛ばされて不便なり。かつまた山高帽は丈夫にて雨にあたりても形崩れず、甚経済なるものなり。夏の炎天にても黒山高帽にてすこしも可笑しきことなし。中折帽は春より夏にかけて年々の流行あり。されば中折帽を冠るほどなれば洋服もこれに
    <|im_start|>assistant



In [32]:

from tqdm import tqdm
import time
out_file_path = f"data/gen_mult_{current_time_no_symbols}.jsonl"

with open(out_file_path, "a") as f:
    for i in tqdm(range(10**4)):
        try:
            messages = gen_text(genres, model, n_replies=7)
        except Exception as e:
            print(e)
            time.sleep(5)
            continue
        f.write(json.dumps({"messages": messages}, ensure_ascii=False)+"\n")
        time.sleep(1)

'<|im_start|>system\n次の文章をもとに､心温まる日常会話のはじめのセリフを出力しなさい｡対話は敬語で行われます｡セリフのみを出力し、それ以外は何も出力しないでください｡<|im_start|>user\n\n    #参考にする文章: ○帽子は既に述べしが如く洋服の形に従つて各戴くべきものあり。背広に鳥打帽を冠るは適しからず。鳥打帽はその名の如く銃猟、旅行航海等の折にのみ用るものにて、平生都会にてこれを戴くもの巴里あたりにては職工か新聞売子なぞなるべし。欧米ともに黒の山高帽は普通一般に用ひらるるものなり。殊に米国東部の都市にては晴雨共に風甚しきが故、中折帽は吹飛ばされて不便なり。かつまた山高帽は丈夫にて雨にあたりても形崩れず、甚経済なるものなり。夏の炎天にても黒山高帽にてすこしも可笑しきことなし。中折帽は春より夏にかけて年々の流行あり。されば中折帽を冠るほどなれば洋服もこれに\n    <|im_start|>assistant\n'